In [36]:
import re
import pandas as pd

In [37]:
#read dataset 
data = pd.read_csv('train_preprocess.tsv.txt', encoding= 'latin1', names = ['Text','Sentiment'], sep = '\t')
data

,Text,Sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [38]:
#check jumlah data duplikat
data.duplicated().sum()

np.int64(67)

In [39]:
#drop data duplikat 
data = data.drop_duplicates()

In [40]:
#check jumlah data duplikat setelah di drop
data.duplicated().sum()

np.int64(0)

In [41]:
#check missing values 
data.isna().sum()

Text         0
Sentiment    0
dtype: int64

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10933 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       10933 non-null  object
 1   Sentiment  10933 non-null  object
dtypes: object(2)
memory usage: 256.2+ KB


In [43]:
#read kamus alay and abusive
alay = pd.read_csv("new_kamusalay.csv", encoding='latin1', header= None)
alay = alay.rename(columns={0: 'non-alay', 1: 'alay'})              

abuse = pd.read_csv("abusive.csv", encoding='latin1')

In [44]:
#membuat dict kamus alay dan abusive 
alay_dict = dict(zip(alay['non-alay'], alay['alay']))
def fix_alay(text):
    return ' '.join([alay_dict[word] if word in alay_dict else word for word in text.split(' ')])

abuse_dict = dict(zip(abuse['ABUSIVE'], [''] * len(abuse)))
def fix_abusive(text):
    return ' '.join([abuse_dict[word] if word in abuse_dict else word for word in text.split(' ')])

In [45]:
#script untuk data cleansing 
def lower(text):
    return text.lower()
    
def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) 
    text = re.sub('\s+',' ',text)
    text = re.sub('\t+',' ',text)
    text = re.sub('rt',' ',text)
    text = re.sub('user',' ',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    text = re.sub('  +', ' ', text)
    return text

    
def remove_nonaplhanumeric(text):
    text = re.sub(r'\\x[0-9a-fA-F]{2}', '', text)
    text = re.sub(r'\bx[0-9a-fA-F]{1,2}\b', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [46]:
#mengumpulkan semua script tersebut ke satu def 
def cleaning (text):
    text = lower(text)
    text = remove_unnecessary_char(text)
    text = remove_nonaplhanumeric(text)
    text = fix_alay(text)
    text = fix_abusive(text)
    return text

In [49]:
#apply script data cleansing ke dataset 
data ['Text_Bersih']= data['Text'].apply(cleaning)
data = data[['Text','Text_Bersih','Sentiment']]
data.tail(20)

,Text,Text_Bersih,Sentiment
10977,sudah wanita tuna susila mah wanita tuna susil...,sudah wanita tuna susila adalah wanita tuna su...,negative
10978,pdip tidak senang bila nkri damai beragam sih !,partai demokrasi indonesia perjuangan tidak se...,negative
10980,beragam menu bakso tahu dan berbagai menu minu...,beragam menu bakso tahu dan berbagai menu minu...,positive
10981,rumah payung terkenal dengan restoran yang sel...,rumah payung terkenal dengan restoran yang sel...,positive
10982,tolong dibaca pesan saya ! di dalam kotak hand...,tolong dibaca pesan saya di dalam kotak handph...,negative
10983,saya sangat puas dengan spesifikasi laptop asus,saya sangat puas dengan spesifikasi laptop asus,positive
10984,"makanan menurut saya biasa saja , hanya suasan...",makanan menurut saya biasa saja hanya suasana ...,positive
10985,"sumpah gue kecewa banget dengan sms , cuma tea...",sumpah gue kecewa banget dengan sms cuma tease...,negative
10986,"positif : pemandangan nya bagus , menyenangkan...",positif pemandangan nya bagus menyenangkan mat...,positive
10987,ikhlas dalam mengambil hikmah belajar dari keh...,ikhlas dalam mengambil hikmah belajar dari keh...,positive


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10933 entries, 0 to 10999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Text         10933 non-null  object
 1   Text_Bersih  10933 non-null  object
 2   Sentiment    10933 non-null  object
dtypes: object(3)
memory usage: 341.7+ KB
